In [2]:
import pandas as pd

## トレードデータの取得

In [24]:
EURUSD = pd.read_csv('D:\MixBollinger\EURUSD_220710~/MixBollinger_EURUSD.csv')

In [100]:
EURUSD.head()

,SystemDate,Date,DST,Week,OpenTime,CloseTime,OpenPrice,ClosePrice,Type,Profit,MaxProfit,Rate,MaxMinTime,LosCut,Spread,OpenTime(2),CenterTime
0,2010.01.04 10:36,2010.01.04,N,1,10:36,16:19,1.43799,1.44043,0,24.4,69.9,34.9,280,47.213813,1.4,2010/1/4 10:36,2010/1/5 9:04
1,2010.01.05 05:25,2010.01.05,N,2,5:25,9:02,1.44281,1.44404,0,12.3,54.9,22.4,156,14.173674,1.6,2010/1/5 5:25,2010/1/5 9:04
2,2010.01.05 12:05,2010.01.05,N,2,12:05,12:39,1.44097,1.44244,1,-14.7,8.8,-167.0,19,14.771980,1.3,2010/1/5 12:05,2010/1/5 12:39
3,2010.01.05 13:42,2010.01.05,N,2,13:42,14:38,1.44398,1.44258,0,-14.0,14.2,-98.6,5,14.891786,1.4,2010/1/5 13:42,2010/1/5 14:38
4,2010.01.05 15:01,2010.01.05,N,2,15:01,15:14,1.44108,1.44260,1,-15.2,3.6,-422.2,3,15.116385,1.3,2010/1/5 15:01,2010/1/5 15:14


In [5]:
EURUSD.dtypes

SystemDate      object
Date            object
DST             object
Week             int64
OpenTime        object
CloseTime       object
OpenPrice      float64
ClosePrice     float64
Type             int64
Profit         float64
MaxProfit      float64
Rate           float64
MaxMinTime       int64
LosCut         float64
Spread         float64
OpenTime(2)     object
CenterTime      object
dtype: object

## 相場タイプの取得

In [152]:
EURUSD_info = pd.read_excel('D:/07相場情報/相場情報_EURUSD.xlsx', sheet_name="分析", header=1, usecols="B,C,M:P")

In [158]:
EURUSD_info.tail(10)

,date,week,before type,expansion type,after type,expn lineage
3381,2022-12-19,Mon,NaN,NaN,NaN,NaN
3382,2022-12-20,Tue,NaN,NaN,NaN,NaN
3383,2022-12-21,Wed,NaN,NaN,NaN,NaN
3384,2022-12-22,Thu,NaN,NaN,NaN,NaN
3385,2022-12-23,Fri,NaN,NaN,NaN,NaN
3386,2022-12-26,Mon,NaN,NaN,NaN,NaN
3387,2022-12-27,Tue,NaN,NaN,NaN,NaN
3388,2022-12-28,Wed,NaN,NaN,NaN,NaN
3389,2022-12-29,Thu,NaN,NaN,NaN,NaN
3390,2022-12-30,Fri,NaN,NaN,NaN,NaN


In [154]:
EURUSD_info.dtypes

date              datetime64[ns]
week                      object
before type               object
expansion type            object
after type                object
expn lineage              object
dtype: object

## 休日・祝日データ

## 分析データをまとめる

In [155]:
# 日付ごとに集計するための日付リストを作成

# 使用するモジュールのインポート
from datetime import datetime as dt
from datetime import timedelta

# 日付条件の設定
strdt = dt.strptime("2010-01-01 3:00", '%Y-%m-%d %H:%M')  # 開始日
enddt = dt.strptime("2022-07-8 3:00", '%Y-%m-%d %H:%M')  # 終了日

# 日付差の日数を算出（リストに最終日も含めたいので、＋１しています）
days_num = (enddt - strdt).days + 1  # （参考）括弧の部分はtimedelta型のオブジェクトになります

# シンプルにforとappendを使用した場合
analysdata = []
for i in range(days_num):
    dayStart = strdt + timedelta(days=i)

    if dayStart.strftime("%a") == 'Sat' or dayStart.strftime("%a") == 'Sun':      
        continue
    elif dayStart.strftime("%a") == 'Fri':
        dayEnd = strdt + timedelta(days=i+3)
    else: dayEnd = strdt + timedelta(days=i+1)
    
    dateinfo = []
    
    #日付,曜日
    dateinfo.append(dayStart.strftime("%Y.%m.%d"))
    dateinfo.append(dayStart.strftime("%a"))
    
    profit_sum = 0.0
    profit_win = 0.0
    profit_lose = 0.0
    tradeNum_sum = 0
    tradeNum_win = 0
    tradeNum_lose = 0
    profit_max = -999.0
    tradeNum_max = 99999
    for tnum in range(0,len(EURUSD)):
        tradeOpen = dt.strptime(EURUSD.at[tnum,'Date']+ " "+ EURUSD.at[tnum,'OpenTime'],'%Y.%m.%d %H:%M')
        #日にちごとの利益の合計
        if dayStart <= tradeOpen < dayEnd:
            profit = EURUSD.at[tnum,'Profit']
            profit_sum += profit
            tradeNum_sum += 1
            if profit > 0.0:
                profit_win += profit
                tradeNum_win += 1
                
            elif profit < 0.0:
                profit_lose += profit
                tradeNum_lose += 1
                
            if profit_max < profit:
                profit_max = profit
                tradeNum_max = tnum
        elif dayEnd < tradeOpen:
            break
            
    #利益, 利益(勝ち), 利益(負け)     
    dateinfo.append(profit_sum)
    dateinfo.append(profit_win)
    dateinfo.append(profit_lose)
    
    #回数, 回数(勝ち), 回数(負け) , 勝率 
    dateinfo.append(tradeNum_sum)
    dateinfo.append(tradeNum_win)
    dateinfo.append(tradeNum_lose)
    winRate = 0.0
    if tradeNum_sum > 0: winRate = tradeNum_win/tradeNum_sum
    else:  winRate = profit_sum
    dateinfo.append(winRate)
    
    #利益平均, 利益平均(勝ち), 利益平均(負け)
    profAvearge_sum = 0.0
    profAvearge_win = 0.0
    profAvearge_lose = 0.0
    
    if tradeNum_sum>0: profAvearge_sum = profit_sum/tradeNum_sum
    else: profAvearge_sum = profit_sum
    dateinfo.append(profAvearge_sum)
    if tradeNum_win>0: profAvearge_win = profit_win/tradeNum_win
    else: profAvearge_win = profit_win
    dateinfo.append(profAvearge_win)
    if tradeNum_lose>0: profAvearge_lose = profit_lose/tradeNum_lose
    else: profAvearge_lose = profit_lose
    dateinfo.append(profAvearge_lose)
    
    #最大利益,　最大利益トレード番号
    dateinfo.append(profit_max)
    dateinfo.append(tradeNum_max)
        
#    #相場タイプ
#    beforetype = ""
#    expansiontype = ""
#    aftertype = ""
#    expnlineage = ""
#    day = dt.now()
#    for dnum in range(0,len(EURUSD_info)):
#        day = EURUSD_info.at[dnum,'date']
#        if dayStart.strftime("%Y-%m-%d") == day.strftime("%Y-%m-%d"):
#            beforetype = EURUSD_info.at[dnum,'before type']
#            expansiontype = EURUSD_info.at[dnum,'expansion type']
#            aftertype = EURUSD_info.at[dnum,'after type']
#            expnlineage = EURUSD_info.at[dnum,'expn lineage']
#    
#    dateinfo.append(beforetype)
#    dateinfo.append(expansiontype)
#    dateinfo.append(aftertype)
#    dateinfo.append(expnlineage)
    
    
    analysdata.append(dateinfo)
    
    if i % 500 == 0:
        print(dt.now() )
    
analysdata_df = pd.DataFrame(analysdata
     , columns = ['Date'
                  , 'Week'
                  , 'Profit'
                  , 'Profit(W)'
                  , 'Profit(L)'
                  , 'NumTrade'
                  , 'NumTrade(W)'
                  , 'NumTrade(L)'
                  , 'WinRate'
                  , 'ProfAve'
                  , 'ProfAve(W)'
                  , 'ProfAve(L)'
                  , 'Profit(Max)'
                  , 'NumTrade(Max)'
                  #, 'BeforeBollin'
                  #, 'BollinType'
                  #, 'AferBollin'
                  #, 'BollinLineage'
                 ])

2022-08-18 00:08:56.328477
2022-08-18 00:09:16.211490
2022-08-18 00:09:43.759234
2022-08-18 00:11:04.338897
2022-08-18 00:13:02.413082
2022-08-18 00:14:15.133777
2022-08-18 00:15:39.101256
2022-08-18 00:17:16.320556


In [134]:
analysdata_df.dtypes

Date              object
Week              object
Profit           float64
Profit(W)        float64
Profit(L)        float64
NumTrade           int64
NumTrade(W)        int64
NumTrade(L)        int64
WinRate          float64
ProfAve          float64
ProfAve(W)       float64
ProfAve(L)       float64
Profit(Max)      float64
NumTrade(Max)      int64
BeforeBollin      object
BollinType        object
AferBollin        object
BollinLineage     object
dtype: object

In [156]:
analysdata_df

,Date,Week,Profit,Profit(W),Profit(L),NumTrade,NumTrade(W),NumTrade(L),WinRate,ProfAve,ProfAve(W),ProfAve(L),Profit(Max),NumTrade(Max),BeforeBollin,BollinType,AferBollin,BollinLineage
0,2010.01.01,Fri,0.0,0.0,0.0,0,0,0,0.000000,0.000000,0.00,0.000000,-999.0,99999,NaN,NaN,NaN,-
1,2010.01.04,Mon,24.4,24.4,0.0,1,1,0,1.000000,24.400000,24.40,0.000000,24.4,0,range,expansion,squeeze,expansion
2,2010.01.05,Tue,-3.3,55.9,-59.2,6,2,4,0.333333,-0.550000,27.95,-14.800000,43.6,6,squeeze,fake inverse expn,squeeze,expansion
3,2010.01.06,Wed,-23.5,37.2,-60.7,5,1,4,0.200000,-4.700000,37.20,-15.175000,37.2,11,squeeze,expansion,range,expansion
4,2010.01.07,Thu,-6.1,40.3,-46.4,4,2,2,0.500000,-1.525000,20.15,-23.200000,38.8,14,range,expansion,squeeze,expansion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3261,2022.07.04,Mon,-11.2,12.2,-23.4,3,1,2,0.333333,-3.733333,12.20,-11.700000,12.2,14674,squeeze,range,range,range
3262,2022.07.05,Tue,141.0,163.2,-22.2,3,1,2,0.333333,47.000000,163.20,-11.100000,163.2,14678,range,expansion,squeeze,expansion
3263,2022.07.06,Wed,18.9,59.0,-40.1,5,1,4,0.200000,3.780000,59.00,-10.025000,59.0,14683,squeeze,expansion,squeeze,expansion
3264,2022.07.07,Thu,-47.5,16.2,-63.7,6,1,5,0.166667,-7.916667,16.20,-12.740000,16.2,14689,squeeze,trend,trend,expansion


In [159]:
analysdata_df.to_csv('C:/Users/bdffa/OneDrive/FXAnalysisData/analysData_220817.csv')